In [73]:
from Tkinter import *
import tkFileDialog
import sys
import ConfigParser
import os
import logging

In [ ]:
# This looks great!
# http://stackoverflow.com/questions/36604900/redirect-stdout-to-tkinter-text-widget

# also this:
# https://www.blog.pythonlibrary.org/2014/07/14/tkinter-redirecting-stdout-stderr/


# http://stackoverflow.com/questions/14883648/redirect-output-from-python-logger-to-tkinter-widget

In [2]:
# http://stackoverflow.com/questions/36604900/redirect-stdout-to-tkinter-text-widget
class IORedirector(object):
    '''A general class for redirecting I/O to this Text widget.'''
    def __init__(self,text_area):
        self.text_area = text_area
        
#     def flush(self):
#         pass

class StdoutRedirector(IORedirector):
    '''A class for redirecting stdout to this Text widget.'''

    def write(self,str):
        self.text_area.insert("end", str)
        self.text_area.see(END)

    def flush(self):
        pass

In [101]:
class myApp(object):
    def __init__(self, master):
        self.master = master
        master.title('Student Portfolio Populator')

        # get the configuration
        # Path to configuration files
        self.configPath = os.path.expanduser('~/.config/populate/')
        self.configFile = os.path.join(self.configPath, 'config.ini')
        self.getConfig()
        
        self.msgReqURL = 'Paste Google Drive URL for Student Portfolio below:'
        self.msgOKURL = 'Student Portfolio URL accepted:'
        self.message = self.msgReqURL
        
        self.portfolioURL = 'foo'
        self.studentFile = None
        self.studentData = None
        
        
        
        
        self.guiInit()
        
# consider moving this into its own def        
    def guiInit(self):
#         initiate the output window 
#         self.redirector('Student Import Messages:\n')

        # set window size
        self.master.geometry('800x400+30+30')
#         self.label = Label(self.master, text = 'Paste URL for Student Portfolio')

        self.labelURL_text = StringVar()
        self.labelURL_text.set(self.message)
        self.labelURL = Label(self.master, textvariable = self.labelURL_text, font = "TkDefaultFont 0 bold")

        self.labelStuFileRow_text = StringVar()
        self.labelStuFileRow_text.set('Student File: ')
        self.labelStuFileRow = Label(self.master, textvariable = self.labelStuFileRow_text, 
                                     font = "TkDefaultFont 0 bold")

        self.labelStuFile_text = StringVar()
        self.labelStuFile_text.set('None selected')
        self.labelStuFile = Label(self.master, textvariable = self.labelStuFile_text)

        self.submit_button = Button(self.master, text = 'submit', command = self.submit)
        self.quit_button = Button(self.master, text = 'quit', command = self.quit)
        self.buttonStFile = Button(self.master, text = 'set student data file', command = self.studentFilePicker)

        self.entryURL = Entry(self.master, width = 60, validate='key') #, validatecommand=(vcmd, '%P'))
        self.entryURL.insert(END, self.portfolioURL)

        self.labelURL.grid(row = 0, columnspan = 2, sticky=W)        
        self.entryURL.grid(row = 2, column = 0, columnspan = 2, sticky = W)
        self.labelStuFileRow.grid(row = 3, column =0, sticky = W)
        self.labelStuFile.grid(row = 3, column = 1, sticky = W)
        self.buttonStFile.grid(row = 4, column = 0, sticky = W)
        self.submit_button.grid(row = 5, column = 0, sticky = W)
        self.quit_button.grid(row = 6, column = 0, sticky = W)  
        
    def getConfig(self):
        '''
        Reads configuration file and sets the following attributes:
            self.gdBaseFolder
            self.outputFolder
            self.loglevel'''
        
        self.parser = ConfigParser.SafeConfigParser()
        
        # required options in the 'Main' section
        mainSection = 'Main'
        # required key: [method for getting, default value]
        prefs = {'gdBaseFolder': [self.parser.get, 'https://drive.google.com/SAMPLE/URL/abc123zyx987'],
                 'outputFolder': [self.parser.get, os.path.expanduser('~/Desktop/StudentURL.csv')],
                 'loglevel': [self.parser.get, 'INFO']}
        
        # make sure a configuration path exists
        if len(self.parser.read(self.configFile)) <= 0:
            logging.warn('no configuration files found at: {}'.format(self.configFile))
            logging.debug('creating configuration files')
            try:
                os.makedirs(os.path.expanduser(self.configPath))
            except OSError as e:
                if e.errno != 17:
                    logging.critical(e)
                    sys.exit(1)
        
        # make sure there is a main section
        if not self.parser.has_section(mainSection):
            self.parser.add_section(mainSection)
        
        self.preferences = {}
        
        # read search for the expected preferences in the configuration file
        # note which are missing and set to the default values above
        for key in prefs:
            try:
                self.preferences[key] = prefs[key][0](mainSection, key)
            except (ConfigParser.NoSectionError, ConfigParser.NoOptionError):
                self.parser.set(mainSection, key, prefs[key][1])
                self.preferences[key] = prefs[key][1]
        
        self.gdBaseFolder = self.preferences[gdBaseFolder]
        self.outputFolder = self.preferences[outputFolder]
        self.loglevel = self.preferences[loglevel]
        
        
    def studentFilePicker(self):
        try:
            self.studentFile = tkFileDialog.askopenfile(mode='r', title='Choose exported student data')
        except (IOError, OSError) as e:
            print e
            return(False)
        
        
        try:
            self.studentData = self.studentFile.read()
            self.labelStuFile_text.set(self.studentFile.name)
            print self.studentData
        except AttributeError:
            pass
        
        return(True)
    
#     def spawnNew(self):
#         win = Toplevel()
#         win.title('stuff')
#         message = Message(win, text ="some more stuff")
#         message.pack()

    def redirector(self, inputStr=""):
        wX = 800
        wY = 600
        geometry = '{}x{}+40+350'.format(wX, wY)
        import sys
        root = Toplevel()
#         root.lift()
        root.geometry(geometry)
        T = Text(root, font = 'Courier 14', foreground='white', width = wX, height = wY)
        T.config(background='black')
        sys.stdout = StdoutRedirector(T)
        T.pack()
        T.insert(END, inputStr)
        
    def submit(self):
        # spawn a text window for output
       
        self.portfolioURL = self.entryURL.get()
        print self.portfolioURL
        if len(self.portfolioURL)==0:
            self.message = self.msgReqURL
        else:
            # add a check for a valid url here
            self.message = self.msgOKURL
        
        self.labelURL_text.set(self.message)
        
        # write configuration file
        # self.parser.write(open(self.configFile, 'w'))
        # kick off actual task 
        # gDrivePopulate(base folder url, path to credentials etc.)
        
            
        
    def greet(self):
        print('hello world')
        
    def quit(self):
        self.master.destroy()

root = Tk()
my_gui = myApp(root)
root.mainloop()
sys.stdout.flush()

 {'loglevel': 'INFO', 'outputFolder': '/Users/aaronciuffo/Desktop/StudentURL.csv', 'gdBaseFolder': 'https://drive.google.com/SAMPLE/URL/abc123zyx987'}


In [99]:
my_gui.getConfig()


 ['loglevel', 'outputfolder', 'gdbasefolder']


In [69]:
my_gui.parser.read(my_gui.configFile)

[]

In [56]:
my_gui.parser.set('Main', 'basefolder', 'https://drive.google.com/drive/folders/0B9WTleJ1MzaYcmdmTWNNNF9pa1E')
#my_gui.configfile = os.path.expanduser('~/.prefs/populate/')
#my_gui.parser.write(open(my_gui.configfile, 'w'))

In [46]:
prefsPath = '~/.prefs/populate/'
try:
    os.makedirs(os.path.expanduser(prefsPath))
except OSError as e:
    if e.errno != 17:
        print e
    

In [52]:
import errno
errno.errorcode.keys()
errno.errorcode

{1: 'EPERM',
 2: 'ENOENT',
 3: 'ESRCH',
 4: 'EINTR',
 5: 'EIO',
 6: 'ENXIO',
 7: 'E2BIG',
 8: 'ENOEXEC',
 9: 'EBADF',
 10: 'ECHILD',
 11: 'EDEADLK',
 12: 'ENOMEM',
 13: 'EACCES',
 14: 'EFAULT',
 15: 'ENOTBLK',
 16: 'EBUSY',
 17: 'EEXIST',
 18: 'EXDEV',
 19: 'ENODEV',
 20: 'ENOTDIR',
 21: 'EISDIR',
 22: 'EINVAL',
 23: 'ENFILE',
 24: 'EMFILE',
 25: 'ENOTTY',
 26: 'ETXTBSY',
 27: 'EFBIG',
 28: 'ENOSPC',
 29: 'ESPIPE',
 30: 'EROFS',
 31: 'EMLINK',
 32: 'EPIPE',
 33: 'EDOM',
 34: 'ERANGE',
 35: 'EAGAIN',
 36: 'EINPROGRESS',
 37: 'EALREADY',
 38: 'ENOTSOCK',
 39: 'EDESTADDRREQ',
 40: 'EMSGSIZE',
 41: 'EPROTOTYPE',
 42: 'ENOPROTOOPT',
 43: 'EPROTONOSUPPORT',
 44: 'ESOCKTNOSUPPORT',
 45: 'ENOTSUP',
 46: 'EPFNOSUPPORT',
 47: 'EAFNOSUPPORT',
 48: 'EADDRINUSE',
 49: 'EADDRNOTAVAIL',
 50: 'ENETDOWN',
 51: 'ENETUNREACH',
 52: 'ENETRESET',
 53: 'ECONNABORTED',
 54: 'ECONNRESET',
 55: 'ENOBUFS',
 56: 'EISCONN',
 57: 'ENOTCONN',
 58: 'ESHUTDOWN',
 59: 'ETOOMANYREFS',
 60: 'ETIMEDOUT',
 61: 'ECONNREFUS

In [ ]:
root.mainloop()
sys.stdout.flush()

In [ ]:
my_gui.labelURL.cget()

In [ ]:
sys.stdout.flush()
printstuff()